In [1]:
from prep_UD_sents import get_trainable_data, read_ud

In [5]:
pos_map = {
    'ADJ': 0,
    'ADP': 1,
    'ADV': 2,
    'AUX': 3,
    'CCONJ': 4,
    'DET': 5,
    'INTJ': 6,
    'NOUN': 7,
    'NUM': 8,
    'PART': 9,
    'PRON': 10,
    'PROPN': 11,
    'PUNCT': 12,
    'SCONJ': 13,
    'SYM': 14,
    'UNK': 15,
    'VERB': 16,
    'X': 17
}

In [6]:
# store these as parsed texts...
import pickle
import os

train, test, dev = None, None, None
folder = "../data/UD_scrambled_sents_pickles"
os.makedirs(folder, exist_ok=True)
contents = os.listdir(folder)

if not os.path.exists(folder) or len(contents) == 0:
    train = get_trainable_data("train", pos_map)
    test = get_trainable_data("test", pos_map)
    dev = get_trainable_data("dev", pos_map)

    print(f"No files exist in {folder}, dumping pickled files...")
    for name, data in zip(["train", "test", "dev"], [train, test, dev]):
        with open(os.path.join(folder, f"{name}.pickle"), "wb") as f:
            pickle.dump(data, f)
else:
    print(f"Found {len(contents)} files in {folder}, loading pickled files...")
    # load based on names in listdir:
    for content in contents:
        with open(os.path.join(folder, content), "rb") as f:
            data = pickle.load(f)
            if "train" in content:
                train = data
            elif "test" in content:
                test = data
            elif "dev" in content:
                dev = data

Found 3 files in ../data/UD_scrambled_sents_pickles, loading pickled files...


In [7]:
X_train, y_train = train.X, train.y
X_test, y_test = test.X, test.y
X_dev, y_dev = dev.X, dev.y

In [9]:
import tensorflow as tf

MAX_LEN = X_train.shape[1]
print(f"Max len: {MAX_LEN}")
embedding_size = len(pos_map)

l2 = tf.keras.regularizers.l2

model = tf.keras.Sequential([
    tf.keras.layers.Embedding(embedding_size, 32, input_length=MAX_LEN),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(128, return_sequences=True)),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(128)),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(32, activation="relu", kernel_regularizer=l2(0.02)),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(1, activation="sigmoid", kernel_regularizer=l2(0.02))
])

model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])
model.summary()

history = model.fit(X_train, y_train,
                    validation_data=(X_dev, y_dev),
                    epochs=20,
                    batch_size=64
                    )

Max len: 34
Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 34, 32)            576       
                                                                 
 bidirectional_2 (Bidirectio  (None, 34, 256)          164864    
 nal)                                                            
                                                                 
 bidirectional_3 (Bidirectio  (None, 256)              394240    
 nal)                                                            
                                                                 
 dropout_2 (Dropout)         (None, 256)               0         
                                                                 
 dense_2 (Dense)             (None, 32)                8224      
                                                                 
 dropout_3 (Dropout)         (None, 32)   

In [10]:
import random
import spacy
import numpy as np

nlp = spacy.load("nb_core_news_md")

In [11]:
s = "Frakoblingen skjedde ikke, og raketten begynte å spinne. Den spant i omtrent ett minutt før det kom en eksplosjon som ødela begge deler av raketten."
doc = nlp(s)

def pred_from_pos(pos):
    pos = np.pad(pos, (0, MAX_LEN - len(pos)), constant_values="UNK")
    pos = np.vectorize(pos_map.get)(pos)
    pred = model.predict(np.asarray([pos]), verbose=0)[0][0]
    return pred

def pred_from_doc(doc):
    pos = [token.pos_ for token in doc]
    return pred_from_pos(pos)

def pred_from_text(text, nlp):
    doc = nlp(text)
    return pred_from_doc(doc)

pred_from_doc(doc)
        

0.94974697

In [12]:
texts = """
Oppskytningen av raketten «Super Heavy/Starship» feilet. Dette var det første forsøket på å bruke den nye rakett-typen. Raketten skal lande mennesker på månen, og SpaceX har planer om å sende den til Mars.

SpaceX sa kort tid etter eksplosjonen at oppskytningen var delvis velykket. Det på grunn av at raketten ikke eksploderte ved start. Selskapet har samlet mye data som vil informere dem om hva som gikk galt.

Det første tegnet på mulige problemer var synlig allerede 16 sekunder etter start. Tre av motorene i det første trinnet fyrte ikke. Det kommer fram i vidoen fra oppskytningen. 40 sekunder ut i ferden falt én motor til ut. 20 sekunder senere falt den femte motoren ut.

Etter planen skulle bæreraketten Super Heavy skille seg vekk fra romfartøyet Starship omtrent 2 minutter og 40 sekunder etter start.

Frakoblingen skjedde ikke, og raketten begynte å spinne. Den spant i omtrent ett minutt før det kom en eksplosjon som ødela begge deler av raketten.
"""

texts = """
Nordmenn spiser for mye kjøtt
I kroppen vår har vi et hormon som heter insulin. Dette gjør oss i stand til å omdanne sukkeret vi spiser til energi. Når man har diabetes type 2 virker ikke insulinet som det skal, og man får høyt blodsukker fordi mye av sukkeret blir værende i blodet.

Ifølge tall har omtrent 240.000 mennesker i Norge denne livsstilssykdommen. I tillegg regner man med at en del folk lever med sykdommen uten å være klar over det.

I tillegg til dårlig kosthold kan overvekt, inaktivitet, røyking og alkohol utløse sykdommen. Arv er også avgjørende. Har du mor eller far med diabetes type 2, har du selv omtrent 40 prosent sjanse for å utvikle sykdommen i løpet av livet.

Resultatene fra den nye studien viser at man i Norge i 1990 estimerte at 64,9 prosent av tilfellene av sykdommen skyldtes dårlig kosthold. I 2018 hadde dette tallet steget til 75,1 prosent.

Og grunnen til dette? Vi spiser for mye rødt og bearbeidet kjøtt, mener forskerne. I tillegg har vi nordmenn et utilstrekkelig inntak av fullkorn.
"""

In [13]:

def parse_pred(pred, threshold=0.08):
    # badly worded sentences will typically have a score of 0.001 or similar.
    res = "Grammatical" if pred > threshold else "Ungrammatical"
    confidence = pred * 100 if pred > threshold else (1 - pred) * 100
    return f"{res} ({confidence:.0f}% confidence)"

def shuffle_spacy_doc(doc):
    random_seed = 42
    # shuffle on the text and pos tags equally
    texts = [token.text for token in doc]
    pos = [token.pos_ for token in doc]
    random.Random(random_seed).shuffle(texts)
    random.Random(random_seed).shuffle(pos)
    return texts, pos

for text in [t.strip() for t in texts.split("\n") if len(t) > 0]:
    doc = nlp(text)
    for sent in doc.sents:
        pos = [token.pos_ for token in sent]
        # create 3 shuffled examples:
        print(sent)
        print(f"Original: {pos}\n--> {parse_pred(pred_from_pos(pos))}")
        for i in range(3):
            shuffled = pos.copy()
            random.shuffle(shuffled)
            print(f"\tShuffled {i}: {shuffled}")
            print(f"\t--> {parse_pred(pred_from_pos(shuffled))}")
    print("__"*30)
    

Nordmenn spiser for mye kjøtt
Original: ['NOUN', 'VERB', 'ADV', 'ADJ', 'NOUN']
--> Grammatical (55% confidence)
	Shuffled 0: ['NOUN', 'ADJ', 'NOUN', 'ADV', 'VERB']
	--> Ungrammatical (100% confidence)
	Shuffled 1: ['NOUN', 'NOUN', 'VERB', 'ADV', 'ADJ']
	--> Ungrammatical (100% confidence)
	Shuffled 2: ['ADJ', 'VERB', 'NOUN', 'NOUN', 'ADV']
	--> Ungrammatical (100% confidence)
____________________________________________________________
I kroppen vår har vi et hormon som heter insulin.
Original: ['ADP', 'NOUN', 'PRON', 'VERB', 'PRON', 'DET', 'NOUN', 'PRON', 'VERB', 'NOUN', 'PUNCT']
--> Grammatical (93% confidence)
	Shuffled 0: ['ADP', 'PUNCT', 'NOUN', 'PRON', 'VERB', 'PRON', 'NOUN', 'NOUN', 'DET', 'PRON', 'VERB']
	--> Ungrammatical (100% confidence)
	Shuffled 1: ['DET', 'NOUN', 'NOUN', 'PRON', 'PUNCT', 'VERB', 'PRON', 'PRON', 'ADP', 'VERB', 'NOUN']
	--> Ungrammatical (100% confidence)
	Shuffled 2: ['PUNCT', 'PRON', 'VERB', 'NOUN', 'VERB', 'NOUN', 'ADP', 'NOUN', 'DET', 'PRON', 'PRON']
	-

In [15]:
# save tf `model` to disk
os.makedirs("models", exist_ok=True)
from datetime import datetime
timestamp = datetime.now().strftime("%Y%m%d-%H%M%S")
# to file name:
filename = f"models/is_grammatical_{timestamp}.h5"
model.save(filename)